## Costruzione del grafo relativo alla classifica global

In [2]:
from traceback import print_tb
import networkx as nx
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
import pandas as pd

#Credenziali per utilizzare le api della libreria spotipy
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))

#Il file global.csv contiene un link alla classifica top 50 di ognuno dei 72 paesi interessati e alla classifica top 50 globale, oltre a delle informazioni sulla latidudine e longitudine dei paesi
countries = list()
with open('global.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        countries.append(row)

dataset = list()

#scorro tutti i paesi
for country in countries:

    #ottengo la playlist spotify associata al link
    playlist = spotify.playlist(country["Link"])
    #ed estraggo tutte le sue canzoni
    songs = spotify.playlist_items(playlist["id"]) 

    aux = set()

    #il set aux contiene l'ISRC di ogni traccia della playlist
    for i in range (len(songs["items"])):
        aux.add(songs["items"][i]["track"]["external_ids"]["isrc"])
    
    dataset.append(aux)

n = len(dataset)


#Il grafo
grafo = nx.Graph()

#Aggiungo un nodo al grafo per ogni paese interessato, insieme alle informazioni sulla sua latitudine e longitudine (questo sarà utile per usare un plug-in in Gephi che mi sistema i nodi su una cartina geografica)
for i in range (1,len(countries)) :
        grafo.add_node(i,label=countries[i]["Name"],latitudine=float(countries[i]["latitudine"]),longitudine=float(countries[i]["longitudine"]))

#considero ogni coppia di paesi
for x in range (1,n) : 
    for y in range (x+1,n) :
        #calcolo l'affinità tra i due paesi
        affinity = len(dataset[x].intersection(dataset[y]))

        #aggiungo l'arco corrispondente al livello di affinità tra i due paesi se questa non è nulla
        if affinity :
            grafo.add_edge(x,y,weight=affinity)

#infine, salvo il grafo in formato gexf e pkl
nx.write_gexf(grafo,'global15.gexf')
with open('global15.pkl', 'wb') as f:
    pickle.dump(grafo, f)

## Costruzione del grafo relativo alla classifica viral

In [3]:
from traceback import print_tb
import networkx as nx
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
import pandas as pd

#Credenziali per utilizzare le api della libreria spotipy
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))

#Il file global.csv contiene un link alla classifica viral 50 di ognuno dei 72 paesi interessati e alla classifica viral 50 globale, oltre a delle informazioni sulla latidudine e longitudine dei paesi
countries = list()
with open('viral.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        countries.append(row)

dataset = list()

#scorro tutti i paesi
for country in countries:

    #ottengo la playlist spotify associata al link
    playlist = spotify.playlist(country["Link"])
    #ed estraggo tutte le sue canzoni
    songs = spotify.playlist_items(playlist["id"]) 

    aux = set()

    #il set aux contiene l'ISRC di ogni traccia della playlist
    for i in range (len(songs["items"])):
        aux.add(songs["items"][i]["track"]["external_ids"]["isrc"])
    
    dataset.append(aux)

n = len(dataset)


#Il grafo
grafo = nx.Graph()

#Aggiungo un nodo al grafo per ogni paese interessato, insieme alle informazioni sulla sua latitudine e longitudine (questo sarà utile per usare un plug-in in Gephi che mi sistema i nodi su una cartina geografica)
for i in range (1,len(countries)) :
        grafo.add_node(i,label=countries[i]["Name"],latitudine=float(countries[i]["latitudine"]),longitudine=float(countries[i]["longitudine"]))

#considero ogni coppia di paesi
for x in range (1,n) : 
    for y in range (x+1,n) :
        #calcolo l'affinità tra i due paesi
        affinity = len(dataset[x].intersection(dataset[y]))

        #aggiungo l'arco corrispondente al livello di affinità tra i due paesi se questa non è nulla
        if affinity :
            grafo.add_edge(x,y,weight=affinity)

#infine, salvo il grafo in formato gexf e pkl
nx.write_gexf(grafo,'viral15.gexf')
with open('viral15.pkl', 'wb') as f:
    pickle.dump(grafo, f)

## Costruzione del grafo dei generi global

In [ ]:
from traceback import print_tb
import networkx as nx
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
import pandas as pd

#Credenziali per usare le api della libreria spotipy
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))

#Il file global.csv contiene un link alla classifica viral 50 di ognuno dei 72 paesi interessati e alla classifica global 50 globale, oltre a delle informazioni sulla latidudine e longitudine dei paesi
countries = list()
with open('global.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        countries.append(row)

#la lista dei generi, che conterrà dei set corrispondenti ai generi degli artisti che hanno fatto una canzone
dataset = list()

#scorro tutti i paesi
for country in countries:
    
    #ottengo la playlist associata al link
    playlist = spotify.playlist(country["Link"])
    #e ne ottengo tutte le canzoni
    songs = spotify.playlist_items(playlist["id"])
    #print(len(songs))
    aux = set()
    
    #scorro tutte le canzoni ottenute
    for i in range (len(songs["items"])):
        #siccome alcuni artisti non hanno associato un genere, se si prova comunque ad ottenerlo viene sollevata un'eccezione. Per questo racchiudo il codice in un blocco try/catch e, nel caso in cui venisse sollevata, continuo con la prossima canzone
        try: 
            #ottengo i generi dell'artista/artisti che hanno fatto la canzone
            genres = spotify.artist(songs["items"][i]["track"]["artists"][i]["id"])['genres']
            #e li aggiungo tutti nel set aux
            for genre in genres:
                aux.add(genre)
            #aggiungo tutti i generi degli artisti che hanno fatto una canzone nel dataset
            dataset.append(aux)
        except:
            continue   
#print(dataset)

grafo = nx.Graph()

#scorro tutti i generi
for genres in dataset:
    #e considero tutti i generi di un set dentro alla lista per costruire il grafo
    for genre1 in genres:
        for genre2 in genres:
            grafo.add_node(genre1)
            if genre1 != genre2 and not grafo.has_edge(genre2, genre1):
                grafo.add_edge(genre1, genre2)
#salvo il grafo in formato gexf e pkl
nx.write_gexf(grafo, 'global_genres15.gexf')
with open('global_genres15.pkl', 'wb') as f:
    pickle.dump(grafo, f)

## Costruzione del grafo relativo ai generi viral

In [ ]:
from traceback import print_tb
import networkx as nx
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
import pandas as pd

#Credenziali per usare le api della libreria spotipy
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))

#Il file viral.csv contiene un link alla classifica viral 50 di ognuno dei 72 paesi interessati e alla classifica viral 50 globale, oltre a delle informazioni sulla latidudine e longitudine dei paesi
countries = list()
with open('viral.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        countries.append(row)

#la lista dei generi, che conterrà dei set corrispondenti ai generi degli artisti che hanno fatto una canzone
dataset = list()

#scorro tutti i paesi
for country in countries:
    
    #ottengo la playlist associata al link
    playlist = spotify.playlist(country["Link"])
    #e ne ottengo tutte le canzoni
    songs = spotify.playlist_items(playlist["id"])
    #print(len(songs))
    aux = set()
    
    #scorro tutte le canzoni ottenute
    for i in range (len(songs["items"])):
        #siccome alcuni artisti non hanno associato un genere, se si prova comunque ad ottenerlo viene sollevata un'eccezione. Per questo racchiudo il codice in un blocco try/catch e, nel caso in cui venisse sollevata, continuo con la prossima canzone
        try: 
            #ottengo i generi dell'artista/artisti che hanno fatto la canzone
            genres = spotify.artist(songs["items"][i]["track"]["artists"][i]["id"])['genres']
            #e li aggiungo tutti nel set aux
            for genre in genres:
                aux.add(genre)
            #aggiungo tutti i generi degli artisti che hanno fatto una canzone nel dataset
            dataset.append(aux)
        except:
            continue   
#print(dataset)

grafo = nx.Graph()

#scorro tutti i generi
for genres in dataset:
    #e considero tutti i generi di un set dentro alla lista per costruire il grafo
    for genre1 in genres:
        for genre2 in genres:
            grafo.add_node(genre1)
            if genre1 != genre2 and not grafo.has_edge(genre2, genre1):
                grafo.add_edge(genre1, genre2)
#salvo il grafo in formato gexf e pkl
nx.write_gexf(grafo, 'viral_genres15.gexf')
with open('viral_genres15.pkl', 'wb') as f:
    pickle.dump(grafo, f)

## Costruzione dei dataset contenenti le prime 10 canzoni delle playlist top e viral 50 globale

In [ ]:
from traceback import print_tb
import networkx as nx
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
import pandas as pd

#funzione di utilità
def list_toString(lista):
    aux = ""
    
    for el in lista:
        aux += el + ", "
    return aux[:-2]

#credenziali per usare le api della libreria spotipy
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))

#aggiungo tutti i link contenuti nel file global.csv
global_countries = list()
with open('global.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        global_countries.append(row)
        break

#stessa cosa per il file viral.csv
viral_countries = list()
with open('viral.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        viral_countries.append(row)
        break

aux = {}
aux2 = {}

#prendo il link della playlist top 50 global
playlist = spotify.playlist(global_countries[0]["Link"])
#e ne ricavo l'id
songs = spotify.playlist_items(playlist["id"])

#considero le prime 10 canzoni della classifica top 50 globale           
for i in range (10):
    #estraggo isrc, uri e nome della canzone
    isrc = songs["items"][i]["track"]["external_ids"]["isrc"]
    track_uri = songs["items"][i]["track"]["uri"]
    track_name = songs["items"][i]["track"]["name"]

    #lista degli artisti che hanno fatto la canzone
    artist_name = list()
    #aggiungo tutti gli artisti che hanno fatto una canzone
    for j in range(len(songs["items"][i]["track"]["artists"])):
        artist_name.append(songs["items"][i]["track"]["artists"][j]["name"])
    names = list_toString(artist_name)
    album = spotify.album(songs["items"][i]["track"]["album"]["external_urls"]["spotify"])
    data = album["release_date"]
    #aggiungo isrc, uri, titolo, artista/i e data di pubblicazione come info da salvare nel dataset    
    aux[i] = {"isrc":isrc,"uri":track_uri,"titolo":track_name,"artista":names,"pubblicazione":data}


#prendo il link della playlist top 50 viral
playlist = spotify.playlist(viral_countries[0]["Link"])
#dal link estraggo l'id della playlist
songs = spotify.playlist_items(playlist["id"]) 

#e il procedimento è il medesimo fatto prima per la classifica top 50 global
for i in range (10):
    isrc = songs["items"][i]["track"]["external_ids"]["isrc"]

    track_uri = songs["items"][i]["track"]["uri"]
    track_name = songs["items"][i]["track"]["name"]

    artist_name = list()
    for j in range(len(songs["items"][i]["track"]["artists"])):
        artist_name.append(songs["items"][i]["track"]["artists"][j]["name"])
    names = list_toString(artist_name)
    album = spotify.album(songs["items"][i]["track"]["album"]["external_urls"]["spotify"])
    data = album["release_date"]
    
    aux2[i] = {"isrc":isrc,"uri":track_uri,"titolo":track_name,"artista":names,"pubblicazione":data}

#salvo le info ricavate sulle prime 10 canzoni della classifica top 50 global in un csv
with open('15-global.csv', 'w+', newline='', encoding='utf-16') as file:
    writer = csv.DictWriter(file,fieldnames=["isrc","uri","titolo","artista","pubblicazione"])
    writer.writeheader()
    for i in range (10):
        writer.writerow(aux[i])

#salvo le info ricavate sulle prime 10 canzoni della classifica top 50 viral in un csv
with open('15-viral.csv', 'w+', newline='', encoding='utf-16') as file:
    writer = csv.DictWriter(file,fieldnames=["isrc","uri","titolo","artista","pubblicazione"])
    writer.writeheader()
    for i in range (10) :
        writer.writerow(aux2[i])